In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import KeyedVectors
import numpy as np
import spacy 
from transformers import AutoModel, AutoTokenizer
import torch

In [2]:
df=pd.read_excel("Course-co.xlsx")

In [3]:
df

,No,Course Outcomes
0,CO1,to be able to understand the concept of the ra...
1,CO2,to evaluate condition probabilities and condit...
2,CO3,gain the knowledge of applications of discrete...
3,CO4,identify the applications of continuous distri...


In [4]:
course_outcomes=[]
for i in range(len(df)):
    verbs=df[' Course Outcomes'].iloc[i]
    course_outcomes.append(verbs)

In [5]:
course_outcomes

['to be able to understand the concept of the random variable and expectation for discrete and continuous data',
 'to evaluate condition probabilities and conditional expectations',
 'gain the knowledge of applications of discrete distributions in Data Science',
 'identify the applications of continuous distributions in Data Science']

In [6]:
course_verbs=[]
verb=['VB','VBP','VBD','VBG','VBN']
for i in range(len(course_outcomes)):
    review=course_outcomes[i]
    review=review.split()
    review= nltk.pos_tag(review)
    print(review)
    review=[word for word,tag in review if tag in verb]
    course_verbs.append(review)

[('to', 'TO'), ('be', 'VB'), ('able', 'JJ'), ('to', 'TO'), ('understand', 'VB'), ('the', 'DT'), ('concept', 'NN'), ('of', 'IN'), ('the', 'DT'), ('random', 'NN'), ('variable', 'JJ'), ('and', 'CC'), ('expectation', 'NN'), ('for', 'IN'), ('discrete', 'JJ'), ('and', 'CC'), ('continuous', 'JJ'), ('data', 'NNS')]
[('to', 'TO'), ('evaluate', 'VB'), ('condition', 'NN'), ('probabilities', 'NNS'), ('and', 'CC'), ('conditional', 'JJ'), ('expectations', 'NNS')]
[('gain', 'VB'), ('the', 'DT'), ('knowledge', 'NN'), ('of', 'IN'), ('applications', 'NNS'), ('of', 'IN'), ('discrete', 'JJ'), ('distributions', 'NNS'), ('in', 'IN'), ('Data', 'NNP'), ('Science', 'NN')]
[('identify', 'VB'), ('the', 'DT'), ('applications', 'NNS'), ('of', 'IN'), ('continuous', 'JJ'), ('distributions', 'NNS'), ('in', 'IN'), ('Data', 'NNP'), ('Science', 'NN')]


In [7]:
course_verbs

[['be', 'understand'], ['evaluate'], ['gain'], ['identify']]

In [8]:
df_blooms=pd.read_excel("Blooms_index.xlsx")

In [9]:
df_blooms['Verb '] = df_blooms['Verb '].str.split()

In [10]:
df_blooms

,Level:,Verb,Examples of Appropriate Assessments
0,Remembering: can the student recall or remembe...,"[Recall, Recognize, Identify]",Objective test items such as fill-in-the-blank...
1,Understanding: can the student explain ideas o...,"[Interpret, Exemplify, Classify, Summarize, In...","Activities such as papers, exams, problem sets..."
2,Applying: can the student use the information ...,"[Apply, Execute, Implement]","Activities such as problem sets, performances,..."
3,Analyzing: can the student distinguish between...,"[Analyze, Differentiate, Organize, Attribute]","Activities such as case studies, critiques, la..."
4,Evaluating: can the student justify a stand or...,"[Evaluate, Check, Critique, Assess]","Activities such as journals, diaries, critique..."
5,Creating: can the student create new product o...,"[Create, Generate, Plan, Produce, Design]","Activities such as research projects, musical ..."


In [11]:
flattened_course_verbs = [verb[0] for verb in course_verbs if verb]

all_verbs = df_blooms['Verb '].tolist()


In [12]:
all_verbs

[['Recall', 'Recognize', 'Identify'],
 ['Interpret',
  'Exemplify',
  'Classify',
  'Summarize',
  'Infer',
  'Compare',
  'Explain'],
 ['Apply', 'Execute', 'Implement'],
 ['Analyze', 'Differentiate', 'Organize', 'Attribute'],
 ['Evaluate', 'Check', 'Critique', 'Assess'],
 ['Create', 'Generate', 'Plan', 'Produce', 'Design']]

In [13]:
df_blooms['Examples of Appropriate Assessments']

0    Objective test items such as fill-in-the-blank...
1    Activities such as papers, exams, problem sets...
2    Activities such as problem sets, performances,...
3    Activities such as case studies, critiques, la...
4    Activities such as journals, diaries, critique...
5    Activities such as research projects, musical ...
Name: Examples of Appropriate Assessments, dtype: object

## Method-1

In [14]:
flattened_course_verbs = [verb[0] for verb in course_verbs if verb]

# Step 2: Flatten the verbs from the DataFrame
flattened_df_verbs = [' '.join(verbs) for verbs in df_blooms['Verb ']]

# Step 3: Vectorize both course_verbs and df_verbs separately using TF-IDF
vectorizer = TfidfVectorizer()

# Vectorize the flattened course verbs
course_verb_tfidf = vectorizer.fit_transform(flattened_course_verbs)

# Vectorize the flattened DataFrame verbs
df_verb_tfidf = vectorizer.transform(flattened_df_verbs)

# Step 4: Calculate cosine similarity between the course verbs and DataFrame verbs
cosine_sim = cosine_similarity(course_verb_tfidf, df_verb_tfidf)

# Step 5: Retrieve assessments for closely related verbs
matching_assessments = {}
threshold = 0.3  # Set a similarity threshold

for i, course_verb in enumerate(flattened_course_verbs):
    matching_assessments[course_verb] = []
    
    # Compare with the DataFrame verbs
    for j in range(len(df)):
        sim_score = cosine_sim[i, j]  # Get similarity score with the DataFrame verb
        if sim_score > threshold:
            matching_assessments[course_verb].append(df_blooms.iloc[j]['Examples of Appropriate Assessments'])

# Print results
for verb, assessments in matching_assessments.items():
    print(f"Assessments for course verb '{verb}': {assessments}")

Assessments for course verb 'be': []
Assessments for course verb 'evaluate': []
Assessments for course verb 'gain': []
Assessments for course verb 'identify': ['Objective test items such as fill-in-the-blank, matching, labeling, or multiple-choice questions that require students to: recall or recognize terms, facts, and concepts •']


## Method-2

In [15]:
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

In [16]:
# Load BERT model and tokenizer
# model_name = "bert-base-uncased"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModel.from_pretrained(model_name)

flattened_course_verbs2 = [verb[0] for verb in course_verbs if verb]
flattened_df_verbs2 = [' '.join(verbs) for verbs in df_blooms['Verb ']]

# Function to get BERT embedding for a word or phrase
def get_bert_embedding(text):
    try:
        # Tokenize and obtain input IDs
        inputs = tokenizer.encode_plus(
            text,
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=10  # Adjust max_length based on expected input length
        )
        
        # Pass input_ids and attention_mask to the model
        outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])
        
        # Return the mean of the embeddings across tokens
        return outputs.last_hidden_state.mean(dim=1).detach().numpy().flatten()
    except Exception as e:
        print(f"Error processing text '{text}': {e}")
        return None

# Get embeddings for course verbs and DataFrame verbs using BERT
course_verb_vectors2 = np.array([vec for verb in flattened_course_verbs2 
                                 if (vec := get_bert_embedding(verb)) is not None])
df_verb_vectors2 = np.array([vec for verb in flattened_df_verbs2 
                             if (vec := get_bert_embedding(verb)) is not None])

# Check for empty embedding arrays
if course_verb_vectors2.size == 0 or df_verb_vectors2.size == 0:
    print("Error: One or both of the embedding arrays are empty.")
else:
    # Calculate cosine similarity between course verb vectors and DataFrame verb vectors
    cosine_sim2 = cosine_similarity(course_verb_vectors2, df_verb_vectors2)

# Retrieve assessments for closely related verbs
matching_assessments2 = {}
threshold2 = 0.7  # Set a similarity threshold

for i, course_verb in enumerate(flattened_course_verbs2):
    matching_assessments2[course_verb] = []
    if i < len(course_verb_vectors2):  # Check if index is valid
        for j in range(len(df_verb_vectors2)):
            sim_score = cosine_sim2[i, j]
            if sim_score > threshold2:
                matching_assessments2[course_verb].append(df_blooms.iloc[j]['Examples of Appropriate Assessments'])

# Print results
for verb, assessments in matching_assessments2.items():
    print(f"Assessments for course verb '{verb}': {assessments}")


Assessments for course verb 'be': ['Objective test items such as fill-in-the-blank, matching, labeling, or multiple-choice questions that require students to: recall or recognize terms, facts, and concepts •']
Assessments for course verb 'evaluate': ['Objective test items such as fill-in-the-blank, matching, labeling, or multiple-choice questions that require students to: recall or recognize terms, facts, and concepts •', 'Activities such as case studies, critiques, labs, papers, projects, debates, or concept maps that require students to: discriminate or select relevant and irrelevant parts • determine how elements function together • determine bias, values, or underlying intent in presented material •', 'Activities such as journals, diaries, critiques, problem sets, product reviews, or studies that require students to: test, monitor, judge, or critique readings, performances, or products against established • criteria or standards']
Assessments for course verb 'gain': ['Objective tes

## Method-3

In [18]:
# Load SpaCy's pre-trained medium-sized model with word vectors
nlp = spacy.load("en_core_web_md")

# Step 1: Flatten the course verbs
flattened_course_verbs3 = [verb[0] for verb in course_verbs if verb]

# Step 2: Flatten the verbs from the DataFrame
flattened_df_verbs3 = [' '.join(verbs) for verbs in df_blooms['Verb ']]
flattened_df_verbs3

['Recall Recognize Identify',
 'Interpret Exemplify Classify Summarize Infer Compare Explain',
 'Apply Execute Implement',
 'Analyze Differentiate Organize Attribute',
 'Evaluate Check Critique Assess',
 'Create Generate Plan Produce Design']

In [ ]:


# Function to get the SpaCy vector for a word
def get_word_vector(word):
    doc = nlp(word)
    if doc.has_vector:  # Check if the word has a vector
        return doc.vector
    else:
        return np.zeros(nlp.vocab.vectors_length)  # Return a zero vector if word not in vocabulary

# Step 3: Convert course verbs and DataFrame verbs to vectors using SpaCy embeddings
course_verb_vectors3 = np.array([get_word_vector(verb) for verb in flattened_course_verbs3])
df_verb_vectors3 = np.array([get_word_vector(verb) for verb in flattened_df_verbs3])

# Step 4: Calculate cosine similarity between the course verb vectors and DataFrame verb vectors
cosine_sim3 = cosine_similarity(course_verb_vectors3, df_verb_vectors3)

# Step 5: Retrieve assessments for closely related verbs
matching_assessments3 = {}
threshold3 = 0.3  # Set a similarity threshold

for i, course_verb in enumerate(flattened_course_verbs3):
    matching_assessments3[course_verb] = []
    
    # Compare with the DataFrame verbs
    for j in range(len(df_verb_vectors3)):
        sim_score = cosine_sim3[i, j]  # Get similarity score with the DataFrame verb
        if sim_score > threshold3:
            matching_assessments3[course_verb].append(df_blooms.iloc[j]['Examples of Appropriate Assessments'])

# Print results
for verb, assessments in matching_assessments3.items():
    print(f"Verbs:{verb}")
    print(f"Assessments for course verb '{verb}': {assessments}")
